In [1]:
import json
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras

DATA_PATH = "./data.json"


In [2]:
data = json.load(open(DATA_PATH, "r"))

MemoryError: 

In [ ]:
x = np.array(data["mfcc"])
y = np.array(data["labels"])


In [ ]:
# create train, test and validation split
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.1)
x_train, x_validation, y_train, y_validation = train_test_split(
    x_train, y_train, test_size=0.15)

# add an axis to input sets
x_train = x_train[..., np.newaxis]
x_test = x_test[..., np.newaxis]
x_validation = x_validation[..., np.newaxis]

In [ ]:
# create network
input_shape = (x_train.shape[1], x_train.shape[2], 1)
input_shape

In [ ]:
# build network topology
model = keras.Sequential()

# 1st conv layer
model.add(keras.layers.Conv2D(
    32, (3, 3), activation="relu", input_shape=input_shape,))
model.add(keras.layers.MaxPooling2D(
    (3, 3), strides=(2, 2), padding="same"))
model.add(keras.layers.BatchNormalization())

# 2nd conv layer
model.add(keras.layers.Conv2D(32, (3, 3), activation="relu"))
model.add(keras.layers.MaxPooling2D(
    (3, 3), strides=(2, 2), padding="same"))
model.add(keras.layers.BatchNormalization())

# 3rd conv layer
model.add(keras.layers.Conv2D(32, (2, 2), activation="relu"))
model.add(keras.layers.MaxPooling2D(
    (2, 2), strides=(2, 2), padding="same"))
model.add(keras.layers.BatchNormalization())

# flatten output and feed it to dense layer
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(64, activation="relu"))
model.add(keras.layers.Dropout(0.1))

# Output layer
model.add(keras.layers.Dense(10, activation="softmax"))


In [ ]:
# compile model
optimiser = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimiser,
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

In [ ]:
model.summary()

In [ ]:
history = model.fit(x_train, y_train, validation_data=(
    x_validation, y_validation), batch_size=32, epochs=30)

In [ ]:
# plot accuraccy/error for training and validation
fig, axs = plt.subplots(2)

# create accuracy subplot
axs[0].plot(history.history["accuracy"], label="train accuracy")
axs[0].plot(history.history["val_accuracy"], label="test accuracy")
axs[0].set_ylabel("Accuracy")
axs[0].legend(loc="upper right")
axs[0].set_title("Accuracy eval")

# create loss subplot
axs[1].plot(history.history["loss"], label="train loss")
axs[1].plot(history.history["val_loss"], label="test loss")
axs[1].set_ylabel("Loss")
axs[1].legend(loc="lower right")
axs[1].set_title("loss eval")

In [ ]:
# evaluate model on test set
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=1)
print("\nTest accuracy:", test_acc)
print("\nTest loss:", test_loss)